In [1]:
import warnings; warnings.filterwarnings('ignore')

import numpy as np
import oxyba as ox
from importlib import reload; reload(ox);

### Generate correlated random numbers

The first random variable is $Y\sim N(3,0.5)$ and the second $X \sim N(0,0.5)$.
Therefore, the regression model $y = \beta_0 = \beta_1 X + \epsilon$ 
should have residuals $\epsilon \sim N(0,0.5)$ and an intercept $\beta_0 = 3$

In [2]:
N = 1000
np.random.seed(11)
data = np.random.normal(loc=[3.0, 0.0], scale=[0.5, 0.5], size=(N,2))

Make it correlated with $\rho=.25$. 
Thus the regression coefficient should be estimated as $\beta_1 = 0.25$.

In [3]:
data = ox.rand_bivar(data, .25)

A little check what $\rho$ can be computed from this fake dataset

In [4]:
rho,_ = ox.corr(data)
rho = rho[0][1]
print("Beta1 coefficient is expected to be: ", rho)

Beta1 coefficient is expected to be:  0.26653508085149163


### Prepare y and X

Define `y` and `X` in python. Ones are added to `X` for the intercept estimation.

In [5]:
y = data[:,0]
X = np.c_[np.ones((len(y),1)), data[:,1]]

### Estimate Parameters

Run the MLE estimation. The results looks similar to what is expected.

In [6]:
beta = ox.linreg_mle(y,X)
print(beta)

[2.79380104 0.27366747]


Let's compare it againts OLS regression.

In [7]:
print(ox.linreg_ols_lu(y,X))

[2.7938177  0.27367108]


### Check the Residuals
The function `linreg_mle` allows to return the scipy result object. 
The standard deviation of the residuals is one parameter of the MLE estimation 
but it's returned along the regression coefficients.

In [8]:
results = ox.linreg_mle(y,X, debug=True)
print("beta:", results.x[:-1].round(4))
print("sd:  ", results.x[-1].round(3) )

beta: [2.7938 0.2737]
sd:   0.495


All `ox.linreg_...` estimation function are returning only the regression coefficients
what is the reason why the residuals standard deviation is not returned by default.
This metric can be computed from `X` and `beta` outside `linreg_mle` if required.

In [9]:
yhat = ox.linreg_predict(X,beta)
print("sd: ", np.std(yhat-y).round(3) )

sd:  0.495
